## Stable Model Training

## **Colab 環境設置**
修改 - > 筆記本設置 -> 硬件加速器 -> GPU -> 保存

#### NOTES:  
* This is "NoGAN" based training, described in the DeOldify readme.
* This model prioritizes stable and reliable renderings.  It does particularly well on portraits and landscapes.  It's not as colorful as the artistic model.

In [0]:
from os import path
import torch

In [0]:
!git clone https://github.com/start810813/DeOldify

In [0]:
#切換至工作目錄
cd DeOldify

In [0]:
#下載原模型
!wget wget https://www.dropbox.com/s/mwjep3vyqk5mkjc/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen_origin.pth
!wget wget https://www.dropbox.com/s/7a8u20e7xdu1dtd/ColorizeStable_crit.pth?dl=0 -O ./models/ColorizeStable_crit_origin.pth

In [0]:
#安裝所需套件
!pip install -r requirements.txt

In [0]:
!pip install --force-reinstall tb-nightly
#會有版本的錯誤但沒關係，請不要執行restart runtime

In [0]:
#本專案將使用CUDA GPU,裝置需要具有NVIDIA支援CUDA的產品,若是COLAB需設定GPU加速
import os
os.environ['CUDA_VISIBLE_DEVICES']='0' 

In [0]:
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from fasterai.generators import *
from fasterai.critics import *
from fasterai.dataset import *
from fasterai.loss import *
from fasterai.save import *
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageFile

## Setup

In [0]:
#訓練的圖像檔案會放在此路徑中
path = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
path_hr = path
path_lr = path/'bandw'

proj_id = 'StableModel'

gen_name = proj_id + '_gen'
pre_gen_name = gen_name + '_0'
crit_name = proj_id + '_crit'

name_gen = proj_id + '_image_gen'
path_gen = path/name_gen

TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

nf_factor = 2
pct_start = 1e-8

In [0]:
def get_data(bs:int, sz:int, keep_pct:float):
    return get_colorize_data(sz=sz, bs=bs, crappy_path=path_lr, good_path=path_hr, 
                             random_seed=None, keep_pct=keep_pct)

def get_crit_data(classes, bs, sz):
    src = ImageList.from_folder(path, include=classes, recurse=True).random_split_by_pct(0.1, seed=42)
    ll = src.label_from_folder(classes=classes)
    data = (ll.transform(get_transforms(max_zoom=2.), size=sz)
           .databunch(bs=bs).normalize(imagenet_stats))
    return data

def create_training_images(fn,i):
    dest = path_lr/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn).convert('LA').convert('RGB')
    img.save(dest)  
    
def save_preds(dl):
    i=0
    names = dl.dataset.items
    
    for b in dl:
        preds = learn_gen.pred_batch(batch=b, reconstruct=True)
        for o in preds:
            o.save(path_gen/names[i].name)
            i += 1
    
def save_gen_images():
    if path_gen.exists(): shutil.rmtree(path_gen)
    path_gen.mkdir(exist_ok=True)
    data_gen = get_data(bs=bs, sz=sz, keep_pct=0.085)
    save_preds(data_gen.fix_dl)
    PIL.Image.open(path_gen.ls()[0])

## Create black and white training images

Only runs if the directory isn't already created.

In [0]:
#這邊會生成黑白照片的訓練集
if not path_lr.exists():
    il = ImageList.from_folder(path_hr)
    parallel(create_training_images, il.items)

## Pre-train generator

#### NOTE
Most of the training takes place here in pretraining for NoGAN.  The goal here is to take the generator as far as possible with conventional training, as that is much easier to control and obtain glitch-free results compared to GAN training.

### 64px

In [0]:
bs=88
sz=64
keep_pct=1.0

In [0]:
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)

In [0]:
learn_gen = gen_learner_wide(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)

In [0]:
#載入之前訓練的模型，進行遷移訓練
!cp ./models/ColorizeStable_gen_origin.pth ./data/imagenet/ILSVRC/Data/CLS-LOC/bandw/models/ColorizeStable_gen_origin.pth

In [0]:
learn_gen = learn_gen.load('ColorizeStable_gen_origin')

In [0]:
#learn_gen.unfreeze()

In [0]:
learn_gen.callback_fns.append(partial(ImageGenTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GenPre'))

In [0]:
learn_gen.fit_one_cycle(1, pct_start=0.8, max_lr=slice(1e-3))

In [0]:
learn_gen.save(pre_gen_name)

In [0]:
learn_gen.unfreeze()

In [0]:
learn_gen.fit_one_cycle(1, pct_start=pct_start,  max_lr=slice(3e-7, 3e-4))

In [0]:
learn_gen.save(pre_gen_name)

### 128px

In [0]:
bs=20
sz=128
keep_pct=1.0

In [0]:
learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)

In [0]:
learn_gen.unfreeze()

In [0]:
learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(1e-7,1e-4))

In [0]:
learn_gen.save(pre_gen_name)

### 192px

In [0]:
bs=8
sz=192
keep_pct=0.50

In [0]:
learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)

In [0]:
learn_gen.unfreeze()

In [0]:
learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(5e-8,5e-5))

In [0]:
learn_gen.save(pre_gen_name)

## Repeatable GAN Cycle

#### NOTE
Best results so far have been based on repeating the cycle below a few times (about 5-8?), until diminishing returns are hit (no improvement in image quality).  Each time you repeat the cycle, you want to increment that old_checkpoint_num by 1 so that new check points don't overwrite the old.  

In [0]:
old_checkpoint_num = 0
checkpoint_num = old_checkpoint_num + 1
gen_old_checkpoint_name = gen_name + '_' + str(old_checkpoint_num)
gen_new_checkpoint_name = gen_name + '_' + str(checkpoint_num)
crit_old_checkpoint_name = crit_name + '_' + str(old_checkpoint_num)
crit_new_checkpoint_name= crit_name + '_' + str(checkpoint_num)

### Save Generated Images

In [0]:
bs=8
sz=192

In [0]:
learn_gen = gen_learner_wide(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor).load(gen_old_checkpoint_name, with_opt=False)

In [0]:
save_gen_images()

### Pretrain Critic

##### Only need full pretraining of critic when starting from scratch.  Otherwise, just finetune!

In [0]:
if old_checkpoint_num == 0:
    bs=27
    sz=128
    learn_gen=None
    gc.collect()
    data_crit = get_crit_data([name_gen, 'test'], bs=bs, sz=sz)
    data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)
    learn_critic = colorize_crit_learner(data=data_crit, nf=256)
    #載入之前的權重做遷移訓練
    !cp ./models/ColorizeStable_crit_origin.pth ./data/imagenet/ILSVRC/Data/CLS-LOC/models/ColorizeStable_crit_origin.pth
    learn_critic = learn_critic.load('ColorizeStable_crit_origin')
    learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))
    learn_critic.fit_one_cycle(6, 1e-3)
    learn_critic.save(crit_old_checkpoint_name)

In [0]:
bs=16
sz=192

In [0]:
data_crit = get_crit_data([name_gen, 'test'], bs=bs, sz=sz)

In [0]:
data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)

In [0]:
learn_critic = colorize_crit_learner(data=data_crit, nf=256).load(crit_old_checkpoint_name, with_opt=False)

In [0]:
learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))

In [0]:
learn_critic.fit_one_cycle(4, 1e-4)

In [0]:
learn_critic.save(crit_new_checkpoint_name)

### GAN

In [0]:
learn_crit=None
learn_gen=None
gc.collect()

In [0]:
lr=2e-5
sz=192
bs=5

In [0]:
data_crit = get_crit_data([name_gen, 'test'], bs=bs, sz=sz)

In [0]:
learn_crit = colorize_crit_learner(data=data_crit, nf=256).load(crit_new_checkpoint_name, with_opt=False)

In [0]:
learn_gen = gen_learner_wide(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor).load(gen_old_checkpoint_name, with_opt=False)

In [0]:
switcher = partial(AdaptiveGANSwitcher, critic_thresh=0.65)
learn = GANLearner.from_learners(learn_gen, learn_crit, weights_gen=(1.0,1.5), show_img=False, switcher=switcher,
                                 opt_func=partial(optim.Adam, betas=(0.,0.9)), wd=1e-3)
learn.callback_fns.append(partial(GANDiscriminativeLR, mult_lr=5.))
learn.callback_fns.append(partial(GANTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GanLearner', visual_iters=100))
learn.callback_fns.append(partial(GANSaveCallback, learn_gen=learn_gen, filename=gen_new_checkpoint_name, save_iters=100))

#### Instructions:  
Find the checkpoint just before where glitches start to be introduced.  This is all very new so you may need to play around with just how far you go here with keep_pct.

In [0]:
learn.data = get_data(sz=sz, bs=bs, keep_pct=0.03)
learn_gen.freeze_to(-1)
learn.fit(1,lr)

In [0]:
!cp data/imagenet/ILSVRC/Data/CLS-LOC/bandw/models/StableModel_gen_0.pth models/ColorizeStable_gen.pth